In [13]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import pickle
from collections import defaultdict

from torch.utils.data import Dataset, DataLoader
from Mmetrics import *

import LTR
import datautil
import permutationgraph
import DTR
import EEL
import PPG
import PL


def df2ds(df_path):
    with open(df_path, 'rb') as f:
        df = pickle.load(f)
    ds = df.to_dict(orient='list')
    for k in ds:
        ds[k] = np.array(ds[k])
    ds['dlr'] = np.concatenate([np.zeros(1), np.where(np.diff(ds['qid'])==1)[0]+1, np.array([ds['qid'].shape[0]])]).astype(int)
    return type('ltr', (object,), ds)


def dict2ds(df_path):
    with open(df_path, 'rb') as f:
        ds = pickle.load(f)
    return type('ltr', (object,), ds)

ds2019 = df2ds('LTR2019.df')
ds2020 = df2ds('LTR2020.df')

def valid_queries(ds):
    dtr, eel = [], []
    groups = np.unique(ds.g)
    for qid in range(ds.dlr.shape[0] - 1):
        s, e = ds.dlr[qid:qid+2]
        lv = ds.lv[s:e]
        g = ds.g[s:e]
        z = False
        for group in groups:
            if lv[g==group].sum() == 0:
                z = True
                break
        if not z:
            dtr.append(qid)
        if len(np.unique(ds.g[s:e])) > 1:
            eel.append(qid)
            
    return {'DTR':np.array(dtr), 'EEL':np.array(eel)}

ds2019.valids = valid_queries(ds2019)
ds2020.valids = valid_queries(ds2020)

def evaluate_one(metric, qid, lv, g, dlr, output_permutation, exposure, sessions_cnt):
    s, e = dlr[qid:qid+2]
    permutation = output_permutation
    lv_s, g_s, sorted_docs_s, dlr_s = \
        EEL.copy_sessions(y=lv[s:e], g=g[s:e], sorted_docs=lv[s:e].argsort()[::-1], sessions=sessions_cnt)
    
    if metric == 'EEL':
        objective_ins = EEL.EEL(y_pred = lv_s, g = g_s, dlr = dlr_s, exposure=exposure, grade_levels = 2)
    else:
        objective_ins = DTR.DTR(y_pred = lv_s, g = g_s, dlr = dlr_s, exposure=exposure)
        
    
    osl = e - s
    argsort = lv[s:e].argsort()[::-1]
    idcg = ((2.**lv[s:e][argsort][:min(osl,10)] - 1.) / (np.log2(2+np.arange(min(osl,10))))).sum()
    ndcg = 0
    for i in range(sessions_cnt):
        tmp = (2.**lv[s:e][permutation[i*osl:(i+1)*osl]-(i*osl)][:min(osl,10)] - 1.)
        ndcg += (tmp / (np.log2(2+np.arange(min(osl,10))))).sum() / idcg
        
    return objective_ins.eval(permutation), ndcg / sessions_cnt

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [36]:

exposure2020 = np.array([1./np.log2(2+i) for i in range(1,np.diff(ds2020.dlr).max()+2)])
exposure2019 = np.array([1./np.log2(2+i) for i in range(1,np.diff(ds2019.dlr).max()+2)])

def learn_one_PPG(metric, qid, alpha, ds, exposure, sessions_cnt):
    s, e = ds.dlr[qid:qid+2]

    PPG_mat = alpha * np.triu(np.ones((e-s,e-s)), 1)
    ref_permutation = ds.y_pred[s:e].argsort()[::-1]
    
    
    output = []
    for i in range(sessions_cnt):
        b = ref_permutation[PPG._PPG_sample(PPG_mat)] + (i * (e-s))
        output.append(b)
    output = np.concatenate(output)
    
    return evaluate_one(metric, qid, ds.lv, ds.g, ds.dlr, output, exposure, sessions_cnt)


def learn_one_PL(metric, qid, alpha, ds, exposure, sessions_cnt):
    s, e = ds.dlr[qid:qid+2]

    log_theta = torch.Tensor(ds.y_pred[s:e]*alpha)
    output = []
    for i in range(sessions_cnt):
        u = torch.distributions.utils.clamp_probs(torch.rand_like(log_theta))
        z = PL.to_z(log_theta, u)
        b = PL.to_b(z) + (i * (e-s))
        output.append(b)
    output = np.concatenate(output)
    
    return evaluate_one(metric, qid, ds.lv, ds.g, ds.dlr, output, exposure, sessions_cnt)


def learn_all(metric, ds, exposure, sessions_cnt, learn_fn, alpha):
    fairs, ndcgs = [], []
    for qid in ds.valids[metric]:
        s,e = ds.dlr[qid:qid+2]

        fair, ndcg = learn_fn(metric, qid, alpha, ds, exposure, sessions_cnt)

        fairs.append(fair)
        ndcgs.append(ndcg)
    return np.array(fairs), np.array(ndcgs)


In [28]:
for alpha in [0.01, 0.2, 0.5, 0.8, 0.9]:
    fair, ndcg = learn_all('EEL', ds2020, exposure2020, 50, learn_one_PPG, alpha)
    print(alpha, fair.mean(), ndcg.mean())

0.01 0.18310315863628504 0.3914338234658324
0.2 0.1796882183899858 0.39059271140253893
0.5 0.17085081917689335 0.38692316658387704
0.8 0.16300707492068617 0.3768847485398579
0.9 0.16285081513862218 0.3664119303960852


In [38]:
for alpha in [0.01, 2, 100, 10000]:
    fair, ndcg = learn_all('EEL', ds2020, exposure2020, 50, learn_one_PL, alpha)
    print(alpha, fair.mean(), ndcg.mean())

0.01 0.09133724640333857 0.35346327447826786
2 0.09220207095782866 0.352870369621071
100 0.1060797356092 0.3755233979247745
10000 0.18058426305563977 0.3911529929785067


In [35]:
validid = 0
ds = ds2020
qid = ds.valids['EEL'][validid]
s, e = ds.dlr[qid:qid+2]
log_theta = torch.Tensor(ds.y_pred[s:e]*100)
output = []
for i in range(4):
    u = torch.distributions.utils.clamp_probs(torch.rand_like(log_theta))
    z = PL.to_z(log_theta, u)
    b = PL.to_b(z)
    output.append(b)
print(output)
print(log_theta.detach().numpy())
ds.y_pred[s:e].argsort()[::-1]

[tensor([ 1,  8,  5,  9,  3,  2,  6,  0,  7, 10,  4]), tensor([ 1,  5,  2,  4,  6,  3,  0, 10,  9,  7,  8]), tensor([ 2,  0,  9,  5,  1,  8,  6,  3, 10,  4,  7]), tensor([ 1,  3,  5,  6,  9,  0,  7,  4,  8,  2, 10])]
[63.468597 64.4381   63.47638  63.202457 62.10881  64.05854  63.718998
 62.15418  61.805176 62.561916 62.358166]


array([ 1,  5,  6,  2,  0,  3,  9, 10,  7,  4,  8])

In [31]:
output

array([ 3,  9,  0,  6,  4,  2,  5,  8,  1,  7, 10,  1,  9, 10,  3,  6,  8,
        4,  0,  5,  7,  2,  1,  9, 10,  4,  2,  3,  5,  7,  8,  6,  0,  8,
        2,  3,  4,  7, 10,  1,  9,  5,  6,  0])

In [ ]:
PPG_mat = alpha * np.triu(np.ones((e-s,e-s)), 1)
ref_permutation = ds.y_pred[s:e].argsort()[::-1]


output = []
for i in range(sessions_cnt):
    b = ref_permutation[PPG._PPG_sample(PPG_mat)] + (i * (e-s))
    output.append(b)
output = np.concatenate(output)

In [49]:
import EEL
import PL
from tqdm.notebook import trange

def learn_one_PL(metric, qid, verbose, y_pred, g, dlr, epochs, lr, exposure, grade_levels, samples_cnt, sessions_cnt):
    s, e = dlr[qid:qid+2]
    
    if metric == 'EEL':
        objective_ins = EEL.EEL(y_pred = y_pred[s:e], g = g[s:e], dlr = np.array([0,e-s]), exposure=exposure, grade_levels = grade_levels)
    else:
        objective_ins = DTR.DTR(y_pred = y_pred[s:e], g = g[s:e], dlr = np.array([0,e-s]), exposure=exposure)
        
    learner = PL.Learner(logits=y_pred[s:e], samples_cnt=samples_cnt, 
                        objective_ins=objective_ins, sessions_cnt=sessions_cnt)
    vals = learner.fit(epochs, lr, verbose=verbose)
    return vals

def learn_all_PL(metric, y_pred, g, dlr, epochs, lr, exposure, grade_levels, samples_cnt, sessions_cnt):
    sorted_docs = []
    
    for qid in trange(dlr.shape[0] - 1, leave=False):
#     for qid in range(dlr.shape[0] - 1):
        min_b = learn_one_PL(metric, qid, 0, y_pred, g, dlr, epochs, lr, exposure, grade_levels, samples_cnt, sessions_cnt)
        sorted_docs.append(min_b)
        

    # print(ndcg_dtr(exposure, lv, np.concatenate(y_rerank), dlr, g, query_counts))
    return sorted_docs

res = learn_all_PL('EEL', ds2020.y_pred, ds2020.g, ds2020.dlr, 50, 0.1, exposure=exposure2020,
        grade_levels=5, samples_cnt=32, sessions_cnt=32)

In [52]:

def evaluate_one(metric, qid, lv, g, dlr, output_permutation, exposure, sessions_cnt):
    s, e = dlr[qid:qid+2]
    permutation = output_permutation[qid]
    lv_s, g_s, sorted_docs_s, dlr_s = \
        EEL.copy_sessions(y=lv[s:e], g=g[s:e], sorted_docs=lv[s:e].argsort()[::-1], sessions=sessions_cnt)
    
    if metric == 'EEL':
        objective_ins = EEL.EEL(y_pred = lv_s, g = g_s, dlr = dlr_s, exposure=exposure, grade_levels = 2)
    else:
        objective_ins = DTR.DTR(y_pred = lv_s, g = g_s, dlr = dlr_s, exposure=exposure)
        
    
    osl = e - s
    argsort = lv[s:e].argsort()[::-1]
    idcg = ((2.**lv[s:e][argsort][:min(osl,10)] - 1.) / (np.log2(2+np.arange(min(osl,10))))).sum()
    ndcg = 0
    for i in range(sessions_cnt):
        ndcg += ((2.**lv[s:e][permutation[i*osl:(i+1)*osl]-(i*osl)][:min(osl,10)] - 1.) / (np.log2(2+np.arange(min(osl,10))))).sum() / idcg
        
    return objective_ins.eval(permutation), ndcg / sessions_cnt
 
def evaluate_all(metric, valids, lv, g, dlr, output_permutation, exposure, sessions_cnt):
    eel_res, eer_res, eed_res, ndcgs = [], [], [], []
#     for qid in range(dlr.shape[0] - 1):
    for qid in valids:
        s,e = dlr[qid:qid+2]
#         if len(np.unique(g[s:e])) == 1:
#             continue
        out1, ndcg = evaluate_one(metric, qid, lv, g, dlr, output_permutation, exposure, sessions_cnt)
#         eel, eer, eed = out1
        eel = out1
        eel_res.append(eel)
#         eer_res.append(eer)
#         eed_res.append(eed)
        ndcgs.append(ndcg)
    return np.array(eel_res), np.array(ndcgs)
#     return np.array(eel_res), np.array(eer_res), np.array(eed_res), np.array(ndcgs)

fair, ndcg = evaluate_all('EEL', ds2020.valids['EEL'], ds2020.lv, ds2020.g, ds2020.dlr, res, exposure2020, sessions_cnt=32)

In [53]:
fair.mean(), ndcg.mean()

(0.17196686498345984, 0.3773136188983286)